### Day 19

### Part 1:
- Scanners can detect relative position of beacons
- Input will be list of scanners and the beacons they can detect
- Positions are in 3D (x,y,z)
- Scanners also don't know their rotation in all 3 axes
    - 24 different orientations
    - (i.e. x axis is in one of 6 directions, then y is in one of 4, and that defines where z is)
- Can reconstruct relative locations of scanners if two scanners have the same 12 beacons detected
- Need to construct the full map of beacons, and count how many there are
- Scanner range is 1000 in each direction

Thoughts:
- Make a scanner class that can map an additional scanner onto it (if at least 12 overlap)
- Then loop through all of the scanners until we have a combined map that works
- Work out the rotation mappings by hand? Or use rotation matrices?

In [1]:
### Misc functions to help coordinate transformations
def sin_cos(angle):
    """Return sin(angle) and cos(angle), for angle in degrees and either 0,90,180,270."""
    if angle == 0:
        return 0,1
    elif angle == 90:
        return 1,0
    elif angle == 180:
        return 0,-1
    elif angle == 270:
        return -1,0

def rotate(coords, alpha, beta, gamma):
    """Rotate one set of coordinates by angles alpha, beta, gamma around the (z,y,x) axes"""
    # Taken from rotation matrix wiki page
    sina,cosa = sin_cos(alpha)
    sinb,cosb = sin_cos(beta)
    sinc,cosc = sin_cos(gamma)

    x,y,z = coords

    newx = cosa*cosb*x + (cosa*sinb*sinc - sina*cosc)*y + (cosa*sinb*cosc + sina*sinc)*z
    newy = sina*cosb*x + (sina*sinb*sinc + cosa*cosc)*y + (sina*sinb*cosc - cosa*sinc)*z
    newz = -sinb*x + cosb*sinc*y + cosb*cosc*z

    return [newx,newy,newz]

def get_all_rotations():
    """Find the unique set of 24 rotations we can make to the coordinates."""
    angles = []
    coords = set()
    for a in [0,90,180,270]:
        for b in [0,90,180,270]:
            for c in [0,90,180,270]:
                output = rotate([1,2,3],a,b,c)
                output = ",".join([str(l) for l in output])
                if output not in coords:
                    coords.add(output)
                    angles.append([a,b,c])
    return angles

In [2]:
x = [1,2,3]
tuple(x)

(1, 2, 3)

In [3]:
from collections import defaultdict
class Scanner(object):
    def __init__(self,beacons,num):

        self.original_beacons = beacons # For rotations
        self.beacons = beacons.copy()
        self.beacon_map = set([tuple(b) for b in beacons])
        self.num = num

    def rotate_beacons(self,angles):
        """Rotate all detected beacons by [alpha,beta,gamma] = angles."""
        alpha,beta,gamma = angles
        
        new_beacons = []
        for b in self.original_beacons:
            new_beacons.append(rotate(b, alpha, beta, gamma))
        
        self.beacons = new_beacons
        
    def align_beacons(self,offset):
        """Loop through beacons and apply an offset [x,y,z]."""
        x,y,z = offset
        new_beacons = []
        for b in self.beacons:
            new_beacons.append([b[0]+x,b[1]+y,b[2]+z])
        self.beacons = new_beacons
        
    def count_overlaps(self,scanner2):
        """Loop through pairs of beacons in each scanner to count how many overlap (with the best offset)."""
        count = 0
        offset_counts = defaultdict(int)
        for b1 in self.beacons:
            for b2 in scanner2.beacons:
                offset = (b1[0]-b2[0], b1[1]-b2[1], b1[2]-b2[2])
                offset_counts[offset] += 1
                    
        # Find the best overlap
        best_counts = 0
        for offset,count in offset_counts.items():
            if count > best_counts:
                best_offset = offset
                best_counts = count
        
        return best_offset, best_counts
    
    def check_for_overlap(self,scanner2, angles):
        """Try some combinations of angles and check if 12 beacons overlap.
        If so, align scanner2 to the coords of this scanner."""
        
        found = False
        offset,counts = 0,0
        a = 0
        for a in angles:
            # Rotate scanner 2
            scanner2.rotate_beacons(a)
            # Check the overlap
            offset, counts = self.count_overlaps(scanner2)
            if counts >= 12:
                found = True
                # Align the beacons
                scanner2.align_beacons(offset)
                break            
                
        return found,offset,a,counts

In [4]:
# Function to read the input and run the actual calculation
def read_input(fname):
    with open(fname, "r") as f:
        data = f.read().splitlines()
        
    scanners = []
    num = 0
    for l in data:
        if l.startswith("---"):
            # New scanner
            beacons = []
            
        elif len(l) == 0:
            # End of scanner
            s = Scanner(beacons,num)
            scanners.append(s)
            num += 1
            
        else:
            # Beacon to add to list
            b = [int(a) for a in l.split(",")]
            beacons.append(b)
            
    return scanners

def align_all_scanners(scanners, verbose=True):
    """Combine a list of scanners so they're all on the same coordinate system"""
    to_compare = scanners[0:1] # Start with this scanner as (0,0,0)
    still_to_match = scanners[1:] # List of scanners still to match
    aligned_to_grid = [] # Scanners we're done with
    n_found = 1 # counter to stop infinite loops
    offsets = [] # For part 2 (order isnt important)
    
    while (len(still_to_match)> 0) and (n_found > 0):
        
        n_found = 0 # Counter to stop infinite loops
        to_compare_next_loop = []

        # Loop through the list of scanners we just aligned
        for s1 in to_compare:
            
            still_to_match_next_loop = []

            # Find any other scanners that can be aligned with this one
            for ix,s2 in enumerate(still_to_match):
                found,offset,a,counts = s1.check_for_overlap(s2,angles)
                
                if found:
                    # This scanner is now aligned
                    if verbose:
                        print("Found a valid coordinate system:",s1.num,s2.num,offset,a, counts)
                    to_compare_next_loop.append(s2)
                    n_found += 1
                    offsets.append(offset)
                else:
                    # If we didnt find a good overlap, then add it to the list for later
                    still_to_match_next_loop.append(s2)
                    
            # Update the list to match
            still_to_match = still_to_match_next_loop
            # And move this scanner to the done column
            aligned_to_grid.append(s1)

        to_compare = to_compare_next_loop
        
    # Finish the list
    aligned_to_grid.extend(to_compare)
    
    if len(still_to_match) >0:
        print("Couldnt overlap",len(still_to_match),"scanners")
    else:
        print("Overlapped all scanners!")
    
    return aligned_to_grid,offsets

def add_all_beacons(scanners):
    """Add the beacons from a set of aligned scanners"""
    beacons = set()
    for s in scanners:
        for b in s.beacons:
            beacons.add(tuple(b))
            
    return beacons

In [5]:
angles = get_all_rotations()

In [6]:
# Test input
scanners = read_input("inputs/day19_test_input.dat")
s,_ = align_all_scanners(scanners,verbose=True)
beacons = add_all_beacons(scanners)
print(len(beacons))

Found a valid coordinate system: 0 1 (68, -1246, -43) [0, 180, 0] 12
Found a valid coordinate system: 1 3 (-92, -2380, -20) [0, 180, 0] 12
Found a valid coordinate system: 1 4 (-20, -1133, 1061) [0, 270, 90] 12
Found a valid coordinate system: 4 2 (1105, -1205, 1229) [0, 180, 270] 12
Overlapped all scanners!
79


In [7]:
# Puzzle input
scanners = read_input("inputs/day19_input.dat")
s,_ = align_all_scanners(scanners,verbose=False)
beacons = add_all_beacons(scanners)
print(len(beacons))

Overlapped all scanners!
400


### Part 2:
- Find the largest manhattan distance between any two scanners

Thoughts:
- Save the offsets in the above code

In [8]:
def find_max_distance(offsets):
    max_distance = 0
    
    n = len(offsets)
    for ix1 in range(n):
        for ix2 in range(ix1+1,n):
            o1 = offsets[ix1]
            o2 = offsets[ix2]
            
            dist = abs(o1[0]-o2[0]) + abs(o1[1]-o2[1]) + abs(o1[2]-o2[2])
            max_distance = max(dist,max_distance)
    return max_distance

In [9]:
# Test input
scanners = read_input("inputs/day19_test_input.dat")
s,offsets = align_all_scanners(scanners,verbose=True)
print(find_max_distance(offsets))

Found a valid coordinate system: 0 1 (68, -1246, -43) [0, 180, 0] 12
Found a valid coordinate system: 1 3 (-92, -2380, -20) [0, 180, 0] 12
Found a valid coordinate system: 1 4 (-20, -1133, 1061) [0, 270, 90] 12
Found a valid coordinate system: 4 2 (1105, -1205, 1229) [0, 180, 270] 12
Overlapped all scanners!
3621


In [10]:
# Puzzle input
scanners = read_input("inputs/day19_input.dat")
s,offsets = align_all_scanners(scanners,verbose=False)
print(find_max_distance(offsets))

Overlapped all scanners!
12168
